In [ ]:
from binance.spot import Spot as Client
import pandas as pd
import numpy as np
import dotenv
import matplotlib.pyplot as plt
import os
from aperture.option_client import OptionMarkPriceClient
import time

In [ ]:
dotenv.load_dotenv()

In [ ]:
client = Client(
    os.getenv("BINANCE_API_KEY"),
    os.getenv("BINANCE_API_SECRET"), 
)

In [ ]:
options_client = OptionMarkPriceClient(
    api_key=os.getenv("BINANCE_API_KEY"),
    secret_key=os.getenv("BINANCE_API_SECRET"),
)

In [ ]:
x_info = options_client.get_exchange_info()

In [ ]:
pd.DataFrame(x_info['optionSymbols']).head(40)

In [ ]:
pd.DataFrame(x_info['optionSymbols'])['expiryDate'].apply(lambda x: str(pd.to_datetime(x, unit='ms').date())).unique()

In [ ]:
x_info.keys()

In [ ]:
expiration_date = "2023-07-14"
coin = 'ETH'
call_chain_df = await options_client.get_option_chain_df(date=expiration_date, coin=coin, option_type='CALL')
put_chain_df = await options_client.get_option_chain_df(date=expiration_date, coin=coin, option_type='PUT')
query_time = pd.Timestamp(time.time() * 1_000_000_000)
call_chain_df

In [ ]:
underlying_price = float(client.ticker_price(symbol=f'{coin}USDT')['price'])

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(
    call_chain_df['strike'],
    call_chain_df['markPrice'],
    color='blue',
    label='Mark Price'
)
plt.ylabel('Mark Price')
plt.legend(loc='upper left')
plt.twinx()
plt.scatter(
    call_chain_df['strike'],
    call_chain_df['askIV'],
    color='black',
    label='bidIV',
    marker="v"
)
plt.scatter(
    call_chain_df['strike'],
    call_chain_df['bidIV'],
    color='black',
    label='bidIV',
    marker="^"
)
plt.plot(
    call_chain_df['strike'],
    call_chain_df['markIV'],
    '--',
    color='black',
    label='markIV'
)
plt.axvline(underlying_price, color='red', label='Underlying Price')
plt.ylabel('IV')
plt.legend()
plt.title("IV and Mark Price for ETH Call Options on 2023-07-14")

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(
    put_chain_df['strike'],
    put_chain_df['markPrice'],
    color='blue',
    label='Mark Price'
)
plt.ylabel('Mark Price')
plt.legend(loc='upper left')
plt.twinx()
plt.scatter(
    put_chain_df['strike'],
    put_chain_df['askIV'],
    color='black',
    label='bidIV',
    marker="v"
)
plt.scatter(
    put_chain_df['strike'],
    put_chain_df['bidIV'],
    color='black',
    label='bidIV',
    marker="^"
)
plt.plot(
    put_chain_df['strike'],
    put_chain_df['markIV'],
    '--',
    color='black',
    label='markIV'
)
plt.axvline(underlying_price, color='red', label='Underlying Price')
plt.ylabel('IV')
plt.legend()
plt.title("IV and Mark Price for ETH Put Options on 2023-07-14")

In [ ]:

otm_put_chain_df = put_chain_df[put_chain_df['strike'] <= underlying_price]
otm_call_chain_df = call_chain_df[call_chain_df['strike'] >= underlying_price]
chain_df = pd.concat([otm_put_chain_df, otm_call_chain_df])
chain_df.sort_values(by='strike', inplace=True)


In [ ]:
chain_df.drop(2, inplace=True)

In [ ]:
(chain_df['bidIV'] + chain_df['askIV']) / 2 - chain_df['markIV']

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(chain_df['strike'], chain_df['markIV'], label='put mark IV', linestyle='--', color='blue')
plt.scatter(put_chain_df['strike'], put_chain_df['askIV'], label='put ask IV', marker='v', color='red')
plt.scatter(put_chain_df['strike'], put_chain_df['bidIV'], label='put bid IV', marker='^', color='red')
plt.scatter(call_chain_df['strike'], call_chain_df['askIV'], label='call ask IV', marker='v', color='green', alpha=0.5)
plt.scatter(call_chain_df['strike'], call_chain_df['bidIV'], label='call bid IV', marker='^', color='green', alpha=0.5)
plt.legend()
plt.grid()
# plt.plot(call_chain_df['strike'], put_chain_df['markIV'], label='put mark IV')

In [ ]:
import numpy as np
from scipy.optimize import curve_fit

def svi_vol(k, a, b, rho, m, sigma):
    return np.sqrt(a + b * (rho * (k - m) + np.sqrt((k - m)**2 + sigma**2)))

strikes = chain_df['strike']
observed_vol = chain_df['markIV']
# Take the log of the strike prices
log_strikes = np.log(strikes)


# Initial parameter guesses: replace these with your actual initial guesses
init_params = np.array(np.array([0.1, 0.1, 0.1, 100, 0.1]))  

tol = 1e-8
ftol = 1e-8
method = 'lm'
# perform the optimization
params_opt, params_cov = curve_fit(svi_vol, log_strikes, observed_vol, p0=init_params, maxfev=10000, method=method, xtol=tol, ftol=tol)

# do a second fit, with parameters close to the first ones
# params_opt, params_cov = curve_fit(svi_vol, log_strikes, observed_vol, p0=params_opt, maxfev=10000, method=method, xtol=tol, ftol=tol)

# do a second fit, with parameters close to the first ones
params_opt, params_cov = curve_fit(svi_vol, log_strikes, observed_vol, p0=params_opt, maxfev=10000, method=method, xtol=tol, ftol=tol)

# print the optimal parameters
print(params_opt)


In [ ]:
plt.figure(figsize=(20, 10))
plt.grid()
plt.plot(strikes, observed_vol, "o", label='emperical mark IV')
plt.plot(strikes, svi_vol(np.log(strikes), *params_opt), "-", label='svi mark IV')
plt.scatter(chain_df['strike'], chain_df['askIV'], label='ask IV', marker='v', color='black')
plt.scatter(chain_df['strike'], chain_df['bidIV'], label='bid IV', marker='^', color='black')
plt.legend()

In [ ]:
import time

In [ ]:
T = pd.Timestamp("2023-07-07 08") - query_time
T / pd.Timedelta(365, unit='D')

In [ ]:


import time 
import numpy as np
from scipy.stats import norm
from scipy.interpolate import interp1d

def black_scholes_call(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

# Let's assume we have these variables (replace with actual values)
S = underlying_price  # Underlying asset price
r = 0  # Risk-free rate
T = (pd.Timestamp("2023-07-07 08") - pd.Timestamp(time.time() * 1_000_000_000)) / pd.Timedelta(365, unit='D')

# Assume we have implied volatilities for these strikes
cont_strikes = np.linspace(1400, 2200, 501)  # The strike prices
observed_vol = svi_vol(np.log(cont_strikes), *params_opt) # The observed implied volatilities

# Calculate option prices for each strike
option_prices = black_scholes_call(S, cont_strikes, T, r, observed_vol)

# Define a function to calculate the second derivative
def second_derivative(x, y):
    first_derivative = np.gradient(y, x)
    return np.gradient(first_derivative, x)

# Calculate the second derivative of the option price with respect to the strike
risk_neutral_pdf = second_derivative(cont_strikes, option_prices)

# Interpolate to create a function for the risk neutral PDF
risk_neutral_pdf_func = interp1d(cont_strikes, risk_neutral_pdf, kind='cubic')

plt.figure(figsize=(20, 10))
plt.plot(cont_strikes, risk_neutral_pdf)


In [ ]:
cont_strikes[1] - cont_strikes[0]

In [ ]:
from scipy.integrate import simps

# Initialize an array for the CDF
cdf = np.zeros_like(risk_neutral_pdf)

# Calculate the CDF at each point in x using Simpson's rule
for i in range(1, len(cont_strikes)):
    cdf[i] = simps(risk_neutral_pdf[:i+1], cont_strikes[:i+1])

# Now cdf is the cumulative distribution function

In [ ]:

plt.plot(cont_strikes, cdf)
plt.grid()
# plt.ylim(0.9, 1.1)

In [ ]:
# plot prices
plt.figure(figsize=(20, 10))
plt.plot(
    chain_df['strike'],
    chain_df['markPrice'],
    'o-',
    label='mark price',
)
plt.plot(
    call_chain_df['strike'],
    call_chain_df['markPrice'],
    'o-',
    label='call mark price',
)
plt.plot(
    put_chain_df['strike'],
    put_chain_df['markPrice'],
    'o-',
    label='call mark price',
)
plt.axvline(underlying_price, color='black', linestyle='--', label='underlying price')
plt.legend()
plt.grid()

In [ ]:
import math
from scipy.stats import norm

def black_scholes_call(S, K, T, r, sigma):
    """
    This function returns the Black-Scholes call price.
    """
    d1 = (math.log(S/K) + (r + sigma**2 / 2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    return S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)

def vega(S, K, T, r, sigma):
    """
    This function returns the Vega of the option, which is needed in the calculation of implied volatility.
    """
    d1 = (math.log(S/K) + (r + sigma**2 / 2) * T) / (sigma * math.sqrt(T))
    return S * norm.pdf(d1) * math.sqrt(T)

def implied_vol_newton_raphson(S, K, T, r, market_price, sigma_init=0.5, max_iter=100, tol=1e-5):
    """
    This function returns the implied volatility using Newton-Raphson method.
    """
    sigma = sigma_init
    for i in range(max_iter):
        price = black_scholes_call(S, K, T, r, sigma)
        v = vega(S, K, T, r, sigma)
        print(v)
        price_diff = price - market_price  # f(sigma)

        # if the difference is very small, break out the loop
        if abs(price_diff) < tol:
            return sigma

        # Newton-Raphson formula
        sigma = sigma - price_diff/v  # sigma - f(sigma)/f'(sigma)

    # If volatility is not found after max iterations, raise an exception
    raise Exception('Implied volatility not found')

# usage example:
S = 100  # Underlying asset price
K = 100  # Strike price
T = 1    # Time to maturity
r = 0.05 # Risk-free interest rate
market_price = 10  # Market price of the option

implied_vol = implied_vol_newton_raphson(S, K, T, r, market_price)
print('Implied Volatility:', implied_vol)


In [ ]:
call_chain_df

In [ ]:
import time

In [ ]:
implied_vol_newton_raphson(
    1868.1, 
    1750, 
    (pd.Timestamp('2023-07-07 08').value / 1_000_000_000  - time.time()) / (3600 * 24 * 365), 
    0, 
    97,
    sigma_init=0.3,
    max_iter=1000
)

In [ ]:
put_chain_df